### For reference later <3

        # self.actions = {
        #     "water": -1,
        #     "sow_seed": 1,
        #     "measure_soil_moisture": 0,
        #     "take_photo": 0,
        #     "suppress_weed": 1
        # }>

In [7]:
import torch
import torch.nn as nn

import torchvision.models as models

from torchvision import transforms

In [6]:

class Actor(nn.Module):
    def __init__(self, hidden_size):
        self.hidden_size = hidden_size
        self.bb = nn.Sequential(
            *(list(models.resnet18(pretrained=True).children())[:-1])
        )
        self.ch = nn.Sequential(
            nn.Linear(512, 512),
            nn.Linear(512, 131072),
            nn.Dropout(0.2)
        )
    
    def forward(self, input_image):
        logits = self.bb(input_image)
        logits = self.ch(logits)
        return logits

In [8]:
class Critic(nn.Module):
    def __init__(self, hidden_size):
        self.hidden_size = hidden_size
        self.bb = nn.Sequential(
            *(list(models.resnet18(pretrained=True).children())[:-1])
        )
        self.ch = nn.Sequential(
            nn.Linear(512, self.hidden_size),
            nn.Linear(self.hidden_size, 1),
            nn.Dropout(0.2)
        )
        
    def forward(self, input_image):
        logits = self.bb(input_image)
        logits  = self.ch(logits)
        return logits

In [9]:
class ActorCritic(nn.Module):
    def __init__(self, actor_hidden_size, critic_hidden_size):
        super().__init__()
        self.actor_hidden_size = actor_hidden_size
        self.critic_hidden_size = critic_hidden_size

        self.Actor = Actor(self.actor_hidden_size)
        self.Critic = Critic(self.critic_hidden_size)
        
    def forward(self, state):
        actor_policy = self.Actor(state)
        critic_score = self.Critic(state)
        return actor_policy, critic_score